In [18]:
# Import Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [33]:
# Load the dataset
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [34]:
# Preprocess  the data
# Drop Irrelevant Features
df = df.drop(['RowNumber','CustomerId','Surname'], axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [35]:
df.shape

(10000, 11)

In [37]:
# Encode categorical variables
encoder_gender = LabelEncoder()
df['Gender'] = encoder_gender.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [39]:
# Encode Multi Label using OHE
ohe_encoder = OneHotEncoder(sparse_output=False, drop='first')
df[ohe_encoder.get_feature_names_out()] = ohe_encoder.fit_transform(df[['Geography']])
df = df.drop('Geography',axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,1.0


In [46]:
ohe_encoder.get_feature_names_out()

array(['Geography_Germany', 'Geography_Spain'], dtype=object)

In [53]:
# Save the encoders
with open('label_encoder_gender.pkl', 'wb') as leg:
    pickle.dump(encoder_gender, leg)
with open('one_hot_encoder_geo.pkl', 'wb') as oheg:
    pickle.dump(ohe_encoder, oheg)

In [54]:
# Select X and y
X = df.loc[:, df.columns != 'Exited']
y = df.loc[:,'Exited']
X.shape, y.shape

((10000, 11), (10000,))

In [55]:
# Split the data into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 11), (2000, 11), (8000,), (2000,))

In [57]:
# Scale the X using standard Scaler
standard_scaler = StandardScaler()
X_train = standard_scaler.fit_transform(X_train)
X_test = standard_scaler.transform(X_test)
X_test

array([[-0.57749609,  0.91324755, -0.6557859 , ..., -1.01960511,
         1.72572313, -0.57638802],
       [-0.29729735,  0.91324755,  0.3900109 , ...,  0.79888291,
        -0.57946723, -0.57638802],
       [-0.52560743, -1.09499335,  0.48508334, ..., -0.72797953,
        -0.57946723,  1.73494238],
       ...,
       [ 0.81311987, -1.09499335,  0.77030065, ..., -1.16591585,
        -0.57946723, -0.57638802],
       [ 0.41876609,  0.91324755, -0.94100321, ..., -0.41163463,
        -0.57946723, -0.57638802],
       [-0.24540869,  0.91324755,  0.00972116, ...,  0.12593183,
         1.72572313, -0.57638802]])

In [58]:
# Save the scaler as pickle
with open('scaler.pkl', 'wb') as ss:
    pickle.dump(standard_scaler, ss)

# ANN Implementation

In [104]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [105]:
# Number of inputs
X_train.shape

(8000, 11)

In [132]:
# Build our ANN model
model = Sequential(
    [
        Input((X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),    # Hidden Layer 2
        Dense(1, activation='sigmoid')    # Output
    ]
)
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [133]:
# Define the optimizer and loss
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [134]:
# Compile the model
model.compile(optimizer=opt, loss=loss, metrics=['f1_score'])

In [135]:
# Setup the tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tf_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [136]:
# Setup Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [137]:
# Train the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[tf_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - f1_score: 0.3365 - loss: 0.4469 - val_f1_score: 0.3285 - val_loss: 0.3603
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - f1_score: 0.3460 - loss: 0.3534 - val_f1_score: 0.3285 - val_loss: 0.3543
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - f1_score: 0.3426 - loss: 0.3488 - val_f1_score: 0.3285 - val_loss: 0.3439
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - f1_score: 0.3517 - loss: 0.3422 - val_f1_score: 0.3285 - val_loss: 0.3529
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - f1_score: 0.3361 - loss: 0.3379 - val_f1_score: 0.3285 - val_loss: 0.3439
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - f1_score: 0.3384 - loss: 0.3397 - val_f1_score: 0.3285 - val_loss: 0.3495
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - f1_score: 0.3358 - loss: 0.3297 - val_f1_score: 0.3285 - val_loss: 0.3455
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - f1_score: 0.3394 - loss: 0.3271 - val_f1_s

In [138]:
# Save the model
model.save('model.h5')

In [139]:
# Load Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [140]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 53552), started 0:38:28 ago. (Use '!kill 53552' to kill it.)

In [141]:
from sklearn.metrics import classification_report

In [142]:
y_pred = model.predict(X_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [143]:
y_pred_class = y_pred.copy()
y_pred_class[y_pred_class<= 0.5] = 0
y_pred_class[y_pred_class> 0.5] = 1
y_pred_class = y_pred_class.reshape(len(y_pred))

In [144]:
y_test

6252    0
4684    0
1731    0
4742    0
4521    0
       ..
6412    1
8285    0
7853    1
1095    1
6929    1
Name: Exited, Length: 2000, dtype: int64

In [145]:
y_pred

array([[0.04343927],
       [0.01518017],
       [0.1324489 ],
       ...,
       [0.6695801 ],
       [0.16910422],
       [0.34749553]], dtype=float32)

In [146]:
y_pred_class

array([0., 0., 0., ..., 1., 0., 0.], dtype=float32)

In [147]:
print(classification_report(y_pred=y_pred_class, y_true=y_test))

              precision    recall  f1-score   support

           0       0.88      0.95      0.92      1607
           1       0.71      0.49      0.58       393

    accuracy                           0.86      2000
   macro avg       0.80      0.72      0.75      2000
weighted avg       0.85      0.86      0.85      2000

